In [32]:
pip install crewai crewai-tools gradio openai arxiv scholarly beautifulsoup4 requests

Note: you may need to restart the kernel to use updated packages.


In [33]:
import os
import gradio as gr
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
import arxiv
from typing import List, Dict
import json


In [34]:
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0.7
)



In [35]:
class ArxivSearchTool:
    def search_papers(self, query: str, max_results: int = 5) -> List[Dict]:
        """Search for papers on arXiv"""
        search = arxiv.Search(
            query=query,
            max_results=max_results,
            sort_by=arxiv.SortCriterion.Relevance
        )

        papers = []
        for result in search.results():
            papers.append({
                "title": result.title,
                "authors": [author.name for author in result.authors],
                "abstract": result.summary,
                "url": result.entry_id,
                "published": str(result.published)
            })
        return papers



In [36]:
arxiv_tool = ArxivSearchTool()


In [37]:
topic_explainer = Agent(
    role='Topic Explainer',
    goal='Explain complex research topics in clear, accessible language and provide comprehensive overviews',
    backstory="""You are an expert educator with deep knowledge across multiple research domains.
    You excel at breaking down complex concepts into understandable explanations while maintaining accuracy.
    You understand the importance of providing context and connecting ideas to help researchers grasp new topics quickly.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

literature_finder = Agent(
    role='Literature Finder',
    goal='Find and summarize relevant research papers, identifying key contributions and methodologies',
    backstory="""You are a seasoned research librarian with expertise in academic literature search.
    You know how to identify the most relevant and impactful papers in any field. You're skilled at
    summarizing papers concisely while capturing their key contributions, methodologies, and findings.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

gap_analyzer = Agent(
    role='Gap Analyzer',
    goal='Identify research gaps, unexplored areas, and suggest promising research directions',
    backstory="""You are a research strategist with years of experience in identifying opportunities
    in academic research. You excel at spotting patterns, finding gaps in existing literature,
    and proposing innovative research directions that could advance the field.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)


In [39]:
def create_tasks(research_topic: str):
    explain_topic_task = Task(
        description=f"""Provide a comprehensive explanation of the research topic: {research_topic}
        Include:
        1. Core concepts and definitions
        2. Historical context and evolution
        3. Current state of the field
        4. Key challenges and open questions
        5. Interdisciplinary connections""",
        agent=topic_explainer,
        expected_output="A detailed explanation of the research topic with all requested components"
    )

    find_literature_task = Task(
        description=f"""Search and analyze literature related to: {research_topic}
        For each paper found:
        1. Provide title, authors, and publication year
        2. Summarize the main contributions
        3. Explain the methodology used
        4. Highlight key findings
        5. Note limitations mentioned by authors
        Use the arxiv_tool to search for papers.""",
        agent=literature_finder,
        expected_output="A comprehensive literature review with summaries of relevant papers"
    )

    analyze_gaps_task = Task(
        description=f"""Based on the topic explanation and literature review for {research_topic}, identify:
        1. Research gaps in the current literature
        2. Unexplored or underexplored areas
        3. Methodological limitations in existing work
        4. Potential interdisciplinary opportunities
        5. Suggest 3-5 specific research questions that could address these gaps
        6. Provide a research outline for the most promising direction""",
        agent=gap_analyzer,
        expected_output="A detailed gap analysis with specific research suggestions and an outline"
    )

    return [explain_topic_task, find_literature_task, analyze_gaps_task]

In [40]:
def create_research_crew(research_topic: str):
    tasks = create_tasks(research_topic)

    crew = Crew(
        agents=[topic_explainer, literature_finder, gap_analyzer],
        tasks=tasks,
        process=Process.sequential,
        verbose=True
    )

    return crew

In [42]:
def run_research_companion(research_topic, specific_focus=None):
    # Combine topic and focus
    full_topic = research_topic
    if specific_focus:
        full_topic += f" with focus on {specific_focus}"

    # Create and run the crew
    crew = create_research_crew(full_topic)

    try:
        # Execute the crew
        result = crew.kickoff()

        # Parse results
        output = {
            "topic": full_topic,
            "explanation": "",
            "literature_review": "",
            "gap_analysis": "",
            "full_output": str(result)
        }

        # Extract individual agent outputs if available
        if hasattr(result, 'tasks_output'):
            if len(result.tasks_output) >= 3:
                output["explanation"] = str(result.tasks_output[0])
                output["literature_review"] = str(result.tasks_output[1])
                output["gap_analysis"] = str(result.tasks_output[2])

        return output

    except Exception as e:
        return {
            "error": f"An error occurred: {str(e)}",
            "topic": full_topic
        }

In [43]:
def gradio_interface(research_topic, specific_focus, api_key):
    if api_key and api_key.strip():
        os.environ["OPENAI_API_KEY"] = api_key
    
    if not research_topic:
        return "Please enter a research topic.", "", "", ""
    
    if not os.getenv("OPENAI_API_KEY"):
        return "Please provide your OpenAI API key.", "", "", ""
    
    results = run_research_companion(research_topic, specific_focus)
    
    if "error" in results:
        error_msg = f"Error: {results['error']}"
        return error_msg, error_msg, error_msg, error_msg
    
    explanation = results.get("explanation", "No explanation generated")
    literature = results.get("literature_review", "No literature review generated") 
    gaps = results.get("gap_analysis", "No gap analysis generated")
    
    full_report = f"""# 🔬 Research Analysis: {results.get('topic', research_topic)}

## 📚 Topic Explanation
{explanation}

## 🔍 Literature Review 
{literature}

## 🎯 Gap Analysis & Research Suggestions
{gaps}
"""
    
    return explanation, literature, gaps, full_report

In [44]:
import os
# Proxy fix
os.environ["NO_PROXY"] = "localhost,127.0.0.1,::1"

# Create Gradio Interface
with gr.Blocks(title="AI Research Paper Companion", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🔬 AI Research Paper Companion
    
    *Powered by AI Agents:* Topic Explainer | Literature Finder | Gap Analyzer
    """)
    
    with gr.Row():
        with gr.Column(scale=2):
            api_key = gr.Textbox(label="🔑 OpenAI API Key", type="password")
            topic = gr.Textbox(label="🎯 Research Topic", placeholder="e.g., Machine Learning, Quantum Computing")
            focus = gr.Textbox(label="🔍 Specific Focus (Optional)", placeholder="e.g., Reinforcement Learning")
            submit = gr.Button("🚀 Start Analysis", variant="primary", size="lg")
    
    with gr.Tab("📊 Agent Results"):
        with gr.Row():
            with gr.Column():
                gr.Markdown("### 📚 Topic Explainer")
                topic_output = gr.Textbox(label="", lines=12, show_label=False)
            
            with gr.Column():
                gr.Markdown("### 🔍 Literature Finder") 
                literature_output = gr.Textbox(label="", lines=12, show_label=False)
            
            with gr.Column():
                gr.Markdown("### 🎯 Gap Analyzer")
                gap_output = gr.Textbox(label="", lines=12, show_label=False)
    
    with gr.Tab("📋 Complete Report"):
        full_output = gr.Markdown("")
    
    submit.click(
        fn=gradio_interface,
        inputs=[topic, focus, api_key],
        outputs=[topic_output, literature_output, gap_output, full_output]
    )

# Launch the interface
demo.launch(
    share=True,
    server_name="0.0.0.0", 
    inbrowser=True,
    debug=True
)

* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://d875a51c887661ce79.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5ea677ff-7cfd-4aa8-84bd-edb024aaba36                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Topic Explainer                                                                                         │
│                                                                                                                 │
│  Task: Provide a comprehensive explanation of the research topic: machine learning                              │
│          Include:                                                                                               │
│          1. Core concepts and definitions                                                                       │
│          2. Historical context and evolution                                                                    │
│          3. Current state of the field                                                                          │
│          4. Key challenges and open questions                                                                   │
│          5. Interdisciplinary connections                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Topic Explainer                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. Core Concepts and Definitions:                                                                              │
│  Machine Learning (ML) is a subfield of Artificial Intelligence (AI) that focuses on the development of         │
│  computer algorithms which can learn from and make decisions or predictions based on data. It consists of       │
│  three primary types: Supervised Learning, where an algorithm learns from labeled data to predict outcomes for  │
│  unforeseen data; Unsupervised Learning, where an algorithm identifies patterns in data without any previous    │
│  labels; and Reinforcement Learning, where an algorithm learns to perform an action from experience.            │
│                                                                                                                 │
│  2. Historical Context and Evolution:                                                                           │
│  The idea of machine learning was born from the aspiration of early computer scientists to create machines      │
│  that could mimic human intelligence. The term "Machine Learning" was first coined by Arthur Samuel in 1959.    │
│  The first successful ML application was Samuel's checkers playing program. During the 1970s and 1980s, the     │
│  development of machine learning was slow due to a lack of computing power and data. However, in the 1990s and  │
│  2000s, with the advent of the internet and significant improvements in computational capabilities, machine     │
│  learning experienced a resurgence. The development of neural networks and deep learning has further            │
│  accelerated the growth of ML.                                                                                  │
│                                                                                                                 │
│  3. Current State of the Field:                                                                                 │
│  Today, ML is a rapidly evolving field, with applications in various domains, from healthcare and finance to    │
│  marketing and entertainment. Deep learning, a subset of ML, is a key driver behind technologies such as        │
│  computer vision, natural language processing, and autonomous vehicles. Furthermore, the rise of big data has   │
│  fueled advancements in ML, providing the vast quantities of data required for training increasingly complex    │
│  models.                                                                                                        │
│                                                                                                                 │
│  4. Key Challenges and Open Questions:                                                                          │
│  Despite the progress, several challenges persist in ML research. Issues like overfitting, where models         │
│  perform well on training data but poorly on new data, and underfitting, where models are too simple to         │
│  capture complex patterns, are common. Additionally, ensuring fairness, accountability, and transparency in ML  │
│  models is a major concern, given their increasing role in decision-making processes. Lastly, the field faces   │
│  the challenge of creating models that can learn from small amounts of data, similar to how humans learn.       │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: cba504d8-5f79-41c0-a136-e6703709200d                                                                     │
│  Agent: Topic Explainer                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Gap Analyzer                                                                                            │
│                                                                                                                 │
│  Task: Based on the topic explanation and literature review for machine learning, identify:                     │
│          1. Research gaps in the current literature                                                             │
│          2. Unexplored or underexplored areas                                                                   │
│          3. Methodological limitations in existing work                                                         │
│          4. Potential interdisciplinary opportunities                                                           │
│          5. Suggest 3-5 specific research questions that could address these gaps                               │
│          6. Provide a research outline for the most promising direction                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Gap Analyzer                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. Research Gaps:                                                                                              │
│    - Despite significant advancements, a key gap lies in the development of ML models that can learn from       │
│  small data sets, resembling human learning.                                                                    │
│    - Ensuring fairness, accountability, and transparency in ML models is also a significant gap, given the      │
│  increasing role of ML in decision-making.                                                                      │
│    - Exploration of how to prevent overfitting and underfitting in ML models is still needed.                   │
│                                                                                                                 │
│  2. Unexplored or Underexplored Areas:                                                                          │
│    - The use of ML in conjunction with fields like neuroscience, psychology, and philosophy is underexplored.   │
│    - Adaptive learning in ML, i.e., models that can learn and adapt in real-time without the need for           │
│  retraining, is a largely unexplored area.                                                                      │
│                                                                                                                 │
│  3. Methodological Limitations:                                                                                 │
│    - Many existing ML models require vast amounts of data for training, which is a significant limitation in    │
│  scenarios where data is scarce or privacy concerns are paramount.                                              │
│    - Current methods often struggle to balance model complexity and generalization, leading to overfitting or   │
│  underfitting.                                                                                                  │
│                                                                                                                 │
│  4. Interdisciplinary Opportunities:                                                                            │
│    - Combining ML with neuroscience could lead to the development of more advanced neural networks inspired by  │
│  the human brain.                                                                                               │
│    - The intersection of ML with psychology could enable the creation of algorithms that better mimic human     │
│  decision-making and perception.                                                                                │
│    - Combining ML with philosophy could lead to novel perspectives on AI ethics and artificial consciousness.   │
│                                                                                                                 │
│  5. Specific Research Questions:                                                                                │
│    - How can machine learning models be developed to learn effectively from smaller data sets?                  │
│    - How can fairness, accountability, and transparency be quantitatively incorporated and measured in ML       │
│  models?                                                                                                        │
│    - How can ML contribute to advancements in neuroscie

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 147b6f9e-6430-420b-b28f-b80cfc7c6282                                                                     │
│  Agent: Gap Analyzer                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5ea677ff-7cfd-4aa8-84bd-edb024aaba36                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: 1. Research Gaps:                                                                                │
│    - Despite significant advancements, a key gap lies in the development of ML models that can learn from       │
│  small data sets, resembling human learning.                                                                    │
│    - Ensuring fairness, accountability, and transparency in ML models is also a significant gap, given the      │
│  increasing role of ML in decision-making.                                                                      │
│    - Exploration of how to prevent overfitting and underfitting in ML models is still needed.                   │
│                                                                                                                 │
│  2. Unexplored or Underexplored Areas:                                                                          │
│    - The use of ML in conjunction with fields like neuroscience, psychology, and philosophy is underexplored.   │
│    - Adaptive learning in ML, i.e., models that can learn and adapt in real-time without the need for           │
│  retraining, is a largely unexplored area.                                                                      │
│                                                                                                                 │
│  3. Methodological Limitations:                                                                                 │
│    - Many existing ML models require vast amounts of data for training, which is a significant limitation in    │
│  scenarios where data is scarce or privacy concerns are paramount.                                              │
│    - Current methods often struggle to balance model complexity and generalization, leading to overfitting or   │
│  underfitting.                                                                                                  │
│                                                                                                                 │
│  4. Interdisciplinary Opportunities:                                                                            │
│    - Combining ML with neuroscience could lead to the development of more advanced neural networks inspired by  │
│  the human brain.                                                                                               │
│    - The intersection of ML with psychology could enable the creation of algorithms that better mimic human     │
│  decision-making and perception.                                                                                │
│    - Combining ML with philosophy could lead to novel perspectives on AI ethics and artificial consciousness.   │
│                                                                                                                 │
│  5. Specific Research Questions:                                                                                │
│    - How can machine learning models be developed to learn effectively from smaller data sets?                  │
│    - How can fairness, accountability, and transparency be quantitatively incorporated and measured in ML       │
│  models?                                              

Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://d875a51c887661ce79.gradio.live
